# Quadratic Voting

In [1]:
import math

import torch
import numpy as np
import pandas as pd

Given a list of sensitivity $\alpha$ to three issues, for example $\alpha = (1.0, 2.0, 3.0)$,
determine how the voter should split its vote among the three issues
$$
x = (x_0, x_1, x_2), \; x_0 + x_1 + x_2 = 1, \; x_0 \geq 0, \; x_1 \geq 0, \; x_2 \geq 0. 
$$
to maximize the associated **utility**
$$
U(x)
:=
\alpha_1 x_1 + \alpha_2 x_2 + \alpha_3 x_3.
$$



In [2]:
alpha = torch.tensor([1.0, 3.0, 2.0])

def U(x):
    "Utility function (supports batching)"
    return torch.tensordot(x, alpha, dims=1)

In [3]:
U(x=torch.tensor([1/3, 1/3, 1/3])) 

tensor(2.)

In [4]:
U(x=torch.tensor([0.0, 1.0, 0.0]))

tensor(3.)

In [5]:
U(torch.tensor([1.0, 0.0, 0.0]))

tensor(1.)

In [6]:
batch = torch.tensor([
    [1/3, 1/3, 1/3],
    [0.0, 1.0, 0.0],
    [1.0, 0.0, 0.0],
])

In [7]:
U(batch)

tensor([2., 3., 1.])

## Unconstrained Optimization

In [8]:
x = torch.tensor([1/3, 1/3, 1/3], requires_grad=True)
optimizer = torch.optim.SGD(params=[x], lr=0.01, maximize=True)

df = []
for i in range(100):
    optimizer.zero_grad()
    utility = U(x)
    x0, x1, x2 = x.detach().numpy()
    df.append({"i": i, "x0": x0, "x1": x1, "x2": x2, "utility": utility.item()})
    utility.backward()
    optimizer.step()

pd.DataFrame(df)

,i,x0,x1,x2,utility
0,0,0.333333,0.333333,0.333333,2.000000
1,1,0.343333,0.363333,0.353333,2.140000
2,2,0.353333,0.393333,0.373333,2.280000
3,3,0.363333,0.423333,0.393333,2.420000
4,4,0.373333,0.453333,0.413333,2.560000
...,...,...,...,...,...
95,95,1.283332,3.183331,2.233332,15.299988
96,96,1.293332,3.213331,2.253332,15.439989
97,97,1.303332,3.243331,2.273332,15.579988
98,98,1.313332,3.273331,2.293332,15.719988


## Projection Methods : kinda hard

In [9]:

## Ahhhaaah we have not forbidden negative votes!!! And clamp and projection do not commute ...

x = torch.tensor([1/3, 1/3, 1/3], requires_grad=True)
optimizer = torch.optim.SGD(params=[x], lr=0.01, maximize=True)

df = []
for i in range(100):
    xp = x - x.mean() + torch.ones(3) / 3
    optimizer.zero_grad()
    utility = U(xp)
    x0, x1, x2 = x.detach().numpy()
    df.append({"i": i, "x0": x0, "x1": x1, "x2": x2, "utility": utility.item()})
    utility.backward()
    optimizer.step()

pd.DataFrame(df)

,i,x0,x1,x2,utility
0,0,0.333333,0.333333,0.333333,2.000000
1,1,0.323333,0.343333,0.333333,2.020000
2,2,0.313333,0.353333,0.333333,2.040000
3,3,0.303333,0.363333,0.333333,2.060000
4,4,0.293333,0.373333,0.333333,2.080000
...,...,...,...,...,...
95,95,-0.616666,1.283332,0.333333,3.899999
96,96,-0.626666,1.293332,0.333333,3.919999
97,97,-0.636666,1.303332,0.333333,3.939999
98,98,-0.646666,1.313332,0.333333,3.959999


## Penalization

In [17]:
### BUGGY ATM

x = torch.tensor([1/3, 1/3, 1/3], requires_grad=True)
mu = 100
optimizer = torch.optim.SGD(params=[x], lr=0.001, maximize=True)

df = []
for i in range(1_000):
    optimizer.zero_grad()
    utility = U(x)
    penalized_utility = utility - mu * ( (x.sum() - 1.0)**2 + min(x[0], 0)**2 + min(x[1], 0)**2 + min(x[2], 0)**2) 
    x0, x1, x2 = x.detach().numpy()
    df.append({"i": i, "x0": x0, "x1": x1, "x2": x2, "utility": utility.item(), "penalized_utility": penalized_utility.item()})
    penalized_utility.backward()
    optimizer.step()

pd.DataFrame(df)

,i,x0,x1,x2,utility,penalized_utility
0,0,0.333333,0.333333,0.333333,3.464102,3.464102
1,1,0.334199,0.335931,0.335065,3.474584,3.471884
2,2,0.334025,0.337480,0.335754,3.479626,3.474357
3,3,0.333438,0.338610,0.336028,3.482507,3.475983
4,4,0.332689,0.339573,0.336138,3.484526,3.477471
...,...,...,...,...,...,...
995,995,0.073921,0.626490,0.308811,3.757834,3.749328
996,996,0.073916,0.626541,0.308766,3.757839,3.749333
997,997,0.073910,0.626591,0.308721,3.757844,3.749338
998,998,0.073905,0.626642,0.308676,3.757849,3.749342


## Reparametrization

Use 3 "free" parameters $(y_0, y_1, y_2)$ in $\mathbb{R}^3$, compute $x$ from them with:

$$
x_i = \frac{\exp y_i}{\exp y_1 + \exp y_2 + \exp y_3}.
$$

By design, $x$ belongs to the allowed parameter sets (⚠️ $x>0$!). 

Optimize wrt $y$.

In [11]:
y = torch.tensor([0.0, 0.0, 0.0], requires_grad=True)
optimizer = torch.optim.SGD(params=[y], lr=0.1, maximize=True)

df = []
for i in range(1_000):
    optimizer.zero_grad()
    x = y.exp() / y.exp().sum() # softmax
    utility = U(x)
    x0, x1, x2 = x.detach().numpy()
    df.append({"i": i, "x0": x0, "x1": x1, "x2": x2, "utility": utility.item()})
    utility.backward()
    optimizer.step()

pd.DataFrame(df)

,i,x0,x1,x2,utility
0,0,0.333333,0.333333,0.333333,2.000000
1,1,0.322286,0.344504,0.333210,2.022218
2,2,0.311495,0.355911,0.332594,2.044416
3,3,0.300967,0.367543,0.331489,2.066576
4,4,0.290707,0.379389,0.329904,2.088682
...,...,...,...,...,...
995,995,0.001779,0.994658,0.003563,2.992878
996,996,0.001778,0.994663,0.003559,2.992886
997,997,0.001776,0.994669,0.003555,2.992893
998,998,0.001774,0.994675,0.003552,2.992901


## Quadratic voting

In [12]:
alpha = torch.tensor([1.0, 3.0, 2.0])

def U(x):
    "Utility function (supports batching)"
    return torch.tensordot(x.sqrt(), alpha, dims=1)

In [13]:
y = torch.tensor([0.0, 0.0, 0.0], requires_grad=True)
optimizer = torch.optim.SGD(params=[y], lr=0.1, maximize=True)

df = []
for i in range(1_000):
    optimizer.zero_grad()
    x = y.exp() / y.exp().sum() # softmax
    utility = U(x)
    x0, x1, x2 = x.detach().numpy()
    df.append({"i": i, "x0": x0, "x1": x1, "x2": x2, "utility": utility.item()})
    utility.backward()
    optimizer.step()

pd.DataFrame(df)

,i,x0,x1,x2,utility
0,0,0.333333,0.333333,0.333333,3.464102
1,1,0.323758,0.343001,0.333241,3.480526
2,2,0.314601,0.352514,0.332885,3.496004
3,3,0.305845,0.361858,0.332297,3.510576
4,4,0.297477,0.371021,0.331503,3.524283
...,...,...,...,...,...
995,995,0.071434,0.642855,0.285711,3.741657
996,996,0.071434,0.642855,0.285711,3.741657
997,997,0.071434,0.642855,0.285711,3.741657
998,998,0.071434,0.642855,0.285711,3.741657


In [14]:
impact = x.detach().sqrt()
impact

tensor([0.2673, 0.8018, 0.5345])

In [15]:
# The impact is proportional to alpha!
norm = torch.linalg.vector_norm
impact / norm(impact) * norm(alpha)

tensor([1.0000, 3.0000, 2.0000])

## Using torch nn modules

In [18]:
class Utility(torch.nn.Module): 
    def __init__(self, alpha):
        super().__init__()
        self.alpha = alpha
        self.logits = torch.nn.parameter.Parameter(torch.tensor([0.0, 0.0, 0.0]))
        self.softmax = Softmax(dim=-1)
    def forward(self):
        return torch.tensordot(self.x, self.alpha, dims=1)
    def get_x(self):
        return self.softmax(self.logits)
    x = property(get_x)

In [ ]:
utility = Utility(torch.tensor([1.0, 3.0, 2.0]))
print(f{"utility.logits)
print(utility.x)
utility()

In [ ]:
optimizer = SGD(utility.parameters(), lr=1.0)

In [ ]:

# OK

utility = Utility([1.0, 3.0, 2.0])
optimizer = SGD(utility.parameters(), lr=1.0)

print(utility.x.detach().numpy())
for i in range(10_000):
    optimizer.zero_grad()
    loss = -utility()
    loss.backward()
    optimizer.step()
    if i % 1_000 == 0:
        print(utility.x.detach().numpy())

In [19]:
class Utility(torch.nn.Module): # minus utility parametrized by logits
    def __init__(self, alpha, quadratic=False):
        super().__init__()
        self.quadratic = quadratic
        self.alpha = alpha
        self.logits = Parameter(rand(3))
        self.softmax = Softmax(dim=-1)
    def forward(self):
        x = self.x
        if self.quadratic:
            x = sqrt(x)
        return x @ self.alpha
    def get_x(self):
        return self.softmax(self.logits)
    x = property(get_x)

In [ ]:

# Here the result is wrong AFAICT, but very stably so!
# I don't get smthin prop to alpha as a result, but prop to the squares of alpha
# What does the theory says then? Yay, that's it, my bad. The effort is prop to alpha square
# and the movement of the needle prop to alpha.

utility = Utility([1.0, 3.0, 2.0], quadratic=True)
optimizer = SGD(utility.parameters(), lr=1.0)

x = utility.x.detach().numpy()
print(x, np.sqrt(x))
for i in range(10_000):
    optimizer.zero_grad()
    loss = -utility()
    loss.backward()
    optimizer.step()
    if i % 1_000 == 0:
        x = utility.x.detach().numpy()
        s = np.sqrt(x)
        print(x, s)

In [ ]:
s[1] / s[0]

In [ ]:
s[2] / s[0]